In [1]:
# Importing the required notebooks
%run DifferentialEvolution.ipynb
%run IslandMethod.ipynb

In [2]:
import os
import sys
import numpy as np
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import Row

from bbobbenchmarks import fgeneric
from bbobbenchmarks import bbobbenchmarks as bn

# Start spark
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('IPDE') \
    .getOrCreate()
sc = spark.sparkContext
    
class Evaluation:

    # Setting the different parameters to evaluate
    _test = {'CR' : '_testCR', 'F' : '_testF'}
    
    
    def __init__(self, title):
        # Setting the CSV title
        self.title = title
        
        # Deleting file if it already exists
        self.filename = "outputs/" + title + ".csv"
        if(os.path.isfile(self.filename)):
            os.remove(self.filename)
            
        # Create the file or open
        #self.file = open(filename, "a")
        
        # Setting header for the CSV
        output_header = "Function,Instance,Dimensions,NP,CR,F,MuSt,CRSt,Local Iterations,"
        output_header += "Workers,Evaluations,Migrations,Global Time,Chromosome,Target,"
        output_header += "Score,Stop Reason"
        self.print_output(output_header)
        
        
    def run(self, test = 'CR', workers = 4, localIt = 40):
        # Setting the functions to test, dimension of individuals and its instan ces
        dimensions = [4]
        function_ids = bn.nfreeIDs 
        instances = range(1, 3)

        # Running all functions
        f = fgeneric.LoggingFunction('tmp')
        values = np.arange(0.1, 1.0, 0.1) if test == 'CR' else np.arange(0.2, 2.0, 0.2)
        for value in values:
            for dim in dimensions:
                for fun_id in function_ids:
                    for iinstance in instances:
                        # Setting new function to evaluate
                        f.setfun(*bn.instantiate(fun_id, iinstance=iinstance))

                        # Evaluation of the functions
                        diffEvoTest = getattr(self, self._test[test])
                        diffEvo = diffEvoTest(f.evalfun, goal = f.ftarget, dim = dim, value = value)
                        island = IslandMethod(diffEvo = diffEvo, workers = workers, localIt = localIt)
                        evaluations = island.execute(sc)

                        # Storing the results obtained
                        elements = [fun_id, iinstance, dim, diffEvo.np, diffEvo.cr, diffEvo.f,
                                    diffEvo.MuSt, diffEvo.CRSt, localIt, workers, evaluations, 
                                    island.migrationCount, island.globalTime, island.get_best_chromosome(), 
                                    f.ftarget, island.get_best_score(), island.stoppingReason]
                        output_data = ",".join(str(e) for e in elements)

                        # Printing output
                        self.print_output(output_data)
        

    def _testCR(self, func, goal, dim, value):
        return DifferentialEvolution(func, goal = goal, d = dim, cr = value)
    
    
    def _testF(self, func, goal, dim, value):
        return DifferentialEvolution(func, goal = goal, d = dim, f = value)
    
        
    def print_output(self, output):
        print(output, file = open(self.filename, "a"))

In [3]:
test = Evaluation("CR-test")
test.run()

[Errno 17] File exists: 'tmp'

Number of Evaluations 20

Initial partition structure: [[(0, (array([ 0.01902421, -4.25718963,  3.63378638,  1.78295113]), 108.15794674238819)), (1, (array([-2.4469894 ,  0.78849032, -2.58601131, -1.60641157]), 106.50086097503669))], [(2, (array([ 1.14587812,  3.34425068,  1.01429994,  0.19682616]), 106.55015807091672)), (3, (array([ 2.42470375, -1.40738997,  2.92036397, -1.19060661]), 107.25708014345264))], [(4, (array([ 0.16648301,  2.91926985,  3.91772641, -2.52854871]), 137.49398819380181)), (5, (array([-3.19915424,  1.93365286,  3.60706513,  2.15597955]), 119.75771884527734))], [(6, (array([ 4.55718406, -3.09755306,  0.66014162, -1.69036116]), 116.77105387832665)), (7, (array([-0.28080652, -1.45626728, -4.86989897, -1.16664142]), 106.60980010743347)), (8, (array([-4.96317258, -2.21600005,  0.61447023, -0.5067807 ]), 115.52014555663717)), (9, (array([ 2.65289813,  0.71888466,  2.69386368,  3.91705431]), 104.40316065602659))], [(10, (array([-2.9822221 


Best chromosome ->: [(18, (array([ 0.25280006, -1.15679612, -0.72400062,  1.92639996]), 79.480000000015465))]

Values after island: [(18, (array([ 0.25280006, -1.15679612, -0.72400062,  1.92639996]), 79.480000000015465)), (15, (array([ 0.25280147, -1.15680737, -0.72399988,  1.92640543]), 79.480000000086008)), (6, (array([ 0.25279735, -1.15678837, -0.72400226,  1.92640077]), 79.480000000147925)), (14, (array([ 0.25279533, -1.15679253, -0.72400907,  1.92640253]), 79.480000000166314)), (11, (array([ 0.25280141, -1.15681559, -0.72399562,  1.9264023 ]), 79.480000000269428)), (10, (array([ 0.25279798, -1.15678966, -0.72401325,  1.92639647]), 79.480000000298816)), (7, (array([ 0.25281425, -1.1567911 , -0.72400235,  1.92641346]), 79.480000000468834)), (16, (array([ 0.25278158, -1.15678885, -0.7240048 ,  1.92640445]), 79.48000000050645)), (2, (array([ 0.25279919, -1.15677676, -0.72400649,  1.92639438]), 79.480000000614538)), (8, (array([ 0.25278794, -1.15677939, -0.7240049 ,  1.92638481]), 79.


Best chromosome ->: [(4, (array([-3.89867234, -2.88998315, -3.80242124,  3.90579601]), 394.48000028680451))]

Values after island: [(4, (array([-3.89867234, -2.88998315, -3.80242124,  3.90579601]), 394.48000028680451)), (0, (array([-3.89840639, -2.88987582, -3.80196176,  3.90516163]), 394.4800006590288)), (12, (array([-3.89815577, -2.89017088, -3.80293281,  3.90644576]), 394.48000111133973)), (8, (array([-3.89799519, -2.88940346, -3.80239841,  3.90477178]), 394.48000184291755)), (16, (array([-3.89854757, -2.89010648, -3.803149  ,  3.90750306]), 394.48000429056566)), (7, (array([-3.89763515, -2.88836686, -3.8012332 ,  3.9038276 ]), 394.4800092214918)), (19, (array([-3.89973766, -2.88836686, -3.80497101,  3.90736084]), 394.48001563365204)), (11, (array([-3.89748538, -2.88764722, -3.80563578,  3.90561052]), 394.48001888468423)), (15, (array([-3.89560001, -2.88726559, -3.8022373 ,  3.90355055]), 394.4800218911783)), (3, (array([-3.8996478 , -2.88902762, -3.79667701,  3.90611377]), 394.480


Best chromosome ->: [(11, (array([ 1.20781017,  0.47597475,  3.54719319,  2.38546669]), -208.54641698961154))]

Values after island: [(11, (array([ 1.20781017,  0.47597475,  3.54719319,  2.38546669]), -208.54641698961154)), (2, (array([ 1.18827912,  0.41275918,  3.57632193,  2.38214667]), -182.8720844828137)), (8, (array([ 1.35545016,  0.45864314,  3.55850998,  2.38059746]), -174.97695532928688)), (18, (array([ 1.17804481,  0.58768628,  3.64452053,  2.38455789]), -117.77508667285724)), (12, (array([ 0.90444637,  0.46344889,  3.55303269,  2.37519278]), -90.839985937569622)), (14, (array([ 1.21520237,  0.57101597,  3.44324545,  2.3888445 ]), -78.817230797513503)), (16, (array([ 1.02654348,  0.46056544,  3.55190132,  2.39848555]), -61.593216485393413)), (10, (array([ 1.18238423,  0.50211203,  3.61191803,  2.37089432]), 77.712093265745239)), (13, (array([ 1.23250353,  0.38827847,  3.56840459,  2.36743409]), 155.31285595061109)), (0, (array([ 1.31574315,  0.45864314,  3.55742434,  2.407133


Values after island 2: [(5, (array([ 1.20716995,  0.44800339,  3.55439988,  2.38639999]), -209.87999999773555)), (13, (array([ 1.20727002,  0.4480072 ,  3.55439938,  2.38640003]), -209.87999998484193)), (15, (array([ 1.20722379,  0.44800418,  3.55439983,  2.38640012]), -209.87999998453736)), (1, (array([ 1.2072707 ,  0.44800793,  3.55440006,  2.38640006]), -209.87999998427847)), (17, (array([ 1.20717108,  0.44801137,  3.55440093,  2.38639999]), -209.87999998010432)), (9, (array([ 1.20712204,  0.44801294,  3.55440052,  2.38639998]), -209.87999997477684)), (2, (array([ 1.20729617,  0.44802027,  3.5543999 ,  2.38640007]), -209.87999994512217)), (6, (array([ 1.20735267,  0.44800755,  3.55440146,  2.38639993]), -209.87999994453875)), (19, (array([ 1.20717649,  0.4480254 ,  3.55440003,  2.38639992]), -209.87999992934377)), (7, (array([ 1.20720319,  0.44800387,  3.55440035,  2.38640028]), -209.87999991452847)), (3, (array([ 1.20724418,  0.44803001,  3.55440056,  2.38640001]), -209.8799999054

...


Values after island 2: [(19, (array([ 2.78067469,  1.11334282,  2.58720916,  3.42809455]), 35.900007916339874)), (15, (array([ 2.77980662,  1.11236573,  2.58714603,  3.42775146]), 35.900021027304952)), (7, (array([ 2.77969867,  1.11203596,  2.58705222,  3.42760208]), 35.900026622583923)), (2, (array([ 2.77906178,  1.11188423,  2.58724436,  3.42755783]), 35.90003875270812)), (8, (array([ 2.77880912,  1.11187743,  2.58723736,  3.42754267]), 35.900049301620527)), (0, (array([ 2.77861006,  1.11163364,  2.58719578,  3.42749104]), 35.90005417341915)), (12, (array([ 2.7784316 ,  1.11161385,  2.58737613,  3.42754772]), 35.900057684786375)), (4, (array([ 2.77822729,  1.11151776,  2.58733078,  3.42749945]), 35.900064136267964)), (11, (array([ 2.7782176 ,  1.11144971,  2.58718811,  3.42739622]), 35.900065245007305)), (3, (array([ 2.77811514,  1.11144971,  2.58713341,  3.42760208]), 35.900069595979993)), (16, (array([ 2.77807399,  1.11120754,  2.58730683,  3.42742634]), 35.90007074795291)), (6, (


Best chromosome ->: [(3, (array([ 2.78067469,  1.11286   ,  2.58725614,  3.42787284]), 35.900006941320996))]

Values after island: [(3, (array([ 2.78067469,  1.11286   ,  2.58725614,  3.42787284]), 35.900006941320996)), (9, (array([ 2.77997693,  1.11236573,  2.58718889,  3.42775146]), 35.900016565357049)), (1, (array([ 2.77969867,  1.11236981,  2.58719419,  3.42779643]), 35.900025223710507)), (17, (array([ 2.77927289,  1.11207683,  2.58724436,  3.4275992 ]), 35.900031394912176)), (5, (array([ 2.77880912,  1.11187743,  2.58725319,  3.42754267]), 35.900049243678609)), (11, (array([ 2.77876591,  1.11144971,  2.58722252,  3.42751313]), 35.900051360266175)), (16, (array([ 2.77861006,  1.11163364,  2.58719578,  3.42749104]), 35.90005417341915)), (18, (array([ 2.7784316 ,  1.11161385,  2.58737613,  3.42754772]), 35.900057684786375)), (12, (array([ 2.77822729,  1.11151776,  2.58740406,  3.42753216]), 35.900064104980636)), (4, (array([ 2.7782176 ,  1.11144971,  2.58718811,  3.42739622]), 35.90


Values after island 2: [(0, (array([ 2.78067469,  1.11286   ,  2.58722426,  3.42787284]), 35.900006848961723)), (6, (array([ 2.78008324,  1.11236573,  2.58709318,  3.42775146]), 35.900015859682021)), (5, (array([ 2.78003479,  1.1128659 ,  2.58730149,  3.42790206]), 35.900015995826557)), (7, (array([ 2.77996361,  1.11276897,  2.58727341,  3.42800234]), 35.900017193796899)), (9, (array([ 2.77975533,  1.11232089,  2.58723242,  3.42772402]), 35.900022891104264)), (15, (array([ 2.77927289,  1.11207683,  2.58724436,  3.4275992 ]), 35.900031394912176)), (16, (array([ 2.77921926,  1.11187743,  2.58728424,  3.42764217]), 35.900033205132715)), (12, (array([ 2.77876591,  1.11171459,  2.58726191,  3.42751313]), 35.900050563215189)), (2, (array([ 2.77861006,  1.11163364,  2.58724584,  3.4275082 ]), 35.900054070742243)), (19, (array([ 2.7784316 ,  1.11161385,  2.58737107,  3.42754772]), 35.90005768444879)), (10, (array([ 2.77822729,  1.11151776,  2.58740406,  3.42753216]), 35.900064104980636)), (14


Best chromosome ->: [(19, (array([ 2.78067469,  1.11286   ,  2.5871401 ,  3.42779515]), 35.900006773918442))]


KeyboardInterrupt: 

In [ ]:
test = Evaluation("F-test")
test.run(test = 'F')